In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("Grazioso Salvare")

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Jillian Ortiz: Project 2 Dashboard'))),
    html.Center(html.B(html.H1('SNHU CS 340: Client/Server Development'))),
    html.Hr(),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'20%','width':'20%'}),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'}
            ]
        )

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable = False,
                         filter_action = "native",
                         sort_action = "native",
                         sort_mode = "multi",
                         column_selectable = False,
                         row_selectable ="single",
                         row_deletable = False,
                         selected_columns = [],
                         selected_rows = [0],
                         page_action = "native",
                         page_current = 0,
                         page_size = 10
                        ),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div([
                # Dropdown menu for pie chart
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'},       
                            {'label': 'Outcome Type', 'value': 'outcome_type'},
                            {'label': 'Sex', 'value': 'sex_upon_outcome'}
                        ],
                        value='breed',
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"10%"}
                ),
                # Pie chart
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    
                ),
                # Geolocation map
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    )
                ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # only return if selected_columns is not none
    if selected_columns:
        
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]


    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns'),
               Output('datatable-id','selected_rows')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    # Code to filter interactive data table with MongoDB queries
    # Query database for characteristics specified by client for each filter type
    # Additionally, exclude euthanized animals
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif filter_type == 'WATER':
        df = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    elif filter_type == 'MOUNTAIN OR WILDERNESS':
        df = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                          "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING':
        df = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                          "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 20},
            "age_upon_outcome_in_weeks":{"$lte": 300},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    # Prepare data and columns to be sent to data table
    data=df.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # Reset selected row to first row when filter setting changes
    selected_rows = [0]
    
    return (data, columns, selected_rows)

# This callback populates the pie chart based on the user's dropdown selection.
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('pie_dropdown', 'value')])
def update_graphs(viewData, dropdownValue):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                names=dropdownValue)
        )    
    ]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]




app.run_server(debug=True)


ModuleNotFoundError: No module named 'animal_shelter'